<a href="https://colab.research.google.com/github/Debojyoti83/Blood-Test-Report-Analysis-Health-Recommendations/blob/main/Blood_Test_Report_Analysis_%26_Health_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Blood Test Report Analysis & Health Recommendations**

In [ ]:
# Install necessary libraries
!pip install crewai
!pip install crewai_tools

In [ ]:
import os
import json
import requests
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool, PDFSearchTool

# Set up API keys
os.environ["GOOGLE_API_KEY"] = "#################################"
os.environ["CUSTOM_SEARCH_ENGINE_ID"] = "####################"
os.environ["OPENAI_API_KEY"] = "dummy_key"  # Set a dummy key to bypass the check because using free api
class CustomGoogleSearchTool:
    """Custom tool to perform Google searches using the Custom Search API."""

    def __init__(self, api_key, cx):
        self.api_key = api_key
        self.cx = cx

    def search(self, query, num_results=5):
        """Performs a search using the Google Custom Search API and returns the results."""
        search_url = "https://www.googleapis.com/customsearch/v1"
        params = {
            'key': self.api_key,
            'cx': self.cx,
            'q': query,
            'num': num_results,
        }
        try:
            response = requests.get(search_url, params=params)
            response.raise_for_status()
            results = response.json()
            return results.get('items', [])
        except requests.RequestException as e:
            print(f"Error during Google search: {e}")
            return []

def create_agents_and_tools():
    """Creates agents and tools needed for the crew."""

    pdf_read_tool = PDFSearchTool()
    file_tool = FileReadTool()
    search_tool = CustomGoogleSearchTool(
        api_key=os.environ["GOOGLE_API_KEY"],
        cx=os.environ["CUSTOM_SEARCH_ENGINE_ID"]
    )

    blood_report_analyst = Agent(
        role='Blood Report Analyst',
        goal='Analyze the given blood test report and summarize key points.',
        backstory='You are an experienced analyst who identifies key health indicators from blood tests.',
        tools=[pdf_read_tool],
        allow_delegation=False,
        verbose=True
    )

    health_advisor = Agent(
        role='Health Advisor',
        goal='Provide health recommendations based on the summary and related articles.',
        backstory='You provide practical health advice by integrating medical data and article findings.',
        tools=[search_tool, file_tool],
        allow_delegation=False,
        verbose=True
    )

    return blood_report_analyst, health_advisor

def create_tasks(blood_report_analyst, health_advisor):
    """Creates tasks for analyzing the blood report and generating health recommendations."""

    analyse_blood_report_task = Task(
        description='Analyze the blood test report and summarize key health indicators and findings.',
        expected_output='A JSON string summarizing the key points from the blood test.',
        agent=blood_report_analyst
    )

    generate_feedback_task = Task(
        description='Provide health recommendations based on the blood report and relevant articles. Include source URLs.',
        expected_output='A JSON string containing health recommendations and relevant article URLs.',
        agent=health_advisor,
        context=[analyse_blood_report_task]
    )

    return analyse_blood_report_task, generate_feedback_task

def main():
    # Create agents and tasks
    blood_report_analyst, health_advisor = create_agents_and_tools()
    analyse_blood_report_task, generate_feedback_task = create_tasks(blood_report_analyst, health_advisor)

    # Assemble the crew
    crew = Crew(
        agents=[blood_report_analyst, health_advisor],
        tasks=[analyse_blood_report_task, generate_feedback_task],
        verbose=2
    )

    # PDF path (input)
    pdf_path = r"C:\Users\sende\Downloads\WM17S.pdf"  # actual PDF path

    # Execute tasks
    try:
        result = crew.kickoff(inputs={'pdf_file': pdf_path})

        # Parse the result (assuming it's a JSON string)
        try:
            parsed_result = json.loads(result)

            # Save the result to a file
            output_path = r"C:\Users\sende\Downloads\health_recommendations.json"  # desired output path
            with open(output_path, 'w') as f:
                json.dump(parsed_result, f, indent=4)
            print(f"Results saved to {output_path}")
        except json.JSONDecodeError:
            print("Error: The result is not a valid JSON string.")
            print("Raw result:", result)
    except Exception as e:
        print(f"An error occurred during execution: {e}")

if __name__ == "__main__":
    main()